In [17]:
import re
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd

In [19]:
url = 'https://www.heimat-der-vorfahren.de/index.php/Thread/6188-Selbst%C3%A4ndige-in-Berlin-1866/?pageNo=1'
html = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

In [20]:
mydivs = soup.find_all('div', class_ = 'messageText')

In [21]:
divs = mydivs.copy()[1:]

In [22]:
strings = []
for br_tag in soup.find_all('br'):
    strings.append(br_tag.next_sibling)


In [23]:
info = [k for k in strings[36:] if 'str' in k]
#print(info)

In [24]:
rows=[['Name1', 'Name2', 'Name3', 'Steet', 'Number', 'Full']]
for string in info:
    
    try:
        name = re.search('\n.*?,',string).group()
        name1_end_idx = re.search('\n.*?,',string, re.IGNORECASE).group().index(',')
        name1 = re.search('\n.*?,', string).group()[1:len(re.search('\n.*?,', string).group()[0:])-1]
        try:
            name2_end_idx = re.search(',.*?,', string, re.IGNORECASE).group()[1:].index(',') +1
            name2 = re.search(',.*?,', string).group()[1:len(re.search(',.*?,', string).group()[0:])-1]
            
            try: 
                name3 = re.search(',.*?,',re.search(',.*,',string).group()[1:]).group()[1:len(re.search(',.*?,',re.search(',.*,',string).group()[1:]).group())-1]
            except:
                name3 = ''
        except: 
            name2 = ''
        
        street_start_idx = len(re.search('\n.*,',string, re.IGNORECASE).group())
        street = re.search('.*?str',string[street_start_idx:]).group()
        
        no = re.search('str.*', string).group()[4:]
        full = string[1:]
        ls = [name1, name2, name3, street, no, full]
        rows.append(ls)
    except:
        name1 = ''
        name2 = ''
        name3 = ''
        street = ''
        no = ''
        ls = [name1, name2, name3, street, no, string]
        
    


In [34]:
with open('entrepreneurs.csv', 'w', newline='') as file:
    writer = csv.writer(file,  delimiter=',', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)

In [30]:
#df = pd.DataFrame(rows)
#df.head()

,0,1,2,3,4,5
0,Name1,Name2,Name3,Steet,Number,Full
1,Remme & Friedmann,,,Alexanderstr,38a,"Remme & Friedmann, Alexanderstr. 38a"
2,Roedel & Vetter,,,Alexanderstr,12,"Roedel & Vetter, Alexanderstr. 12"
3,Sachs,Heinrich,,Alexanderstr,29,"Sachs, Heinrich, Alexanderstr. 29"
4,Fasquel,D.,,Linienstr,119,"Fasquel, D., Linienstr. 119"
